The purpose of this notebook is to be able to import multiple runs at once, and access them either independently, or in-aggregate.

In [1]:
import pandas as pd
import numpy as np
import os
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
INDIR = r'data/csv/'
OUTDIR = r'data/csv/'

files = os.listdir(INDIR)

In [3]:
def get_run_df(file):
    return pd.read_csv(INDIR + file)

In [4]:
def smooth_col(n, col, df):
    ''' n = number of measurements to smooth
        col = column name to apply smoothing to
        df = Pandas dataframe'''

    if len(df) < n:
        print("Reduce smoothing to less than %d" % len(df))
    
    col_name = col + '_smooth_' + str(n)
    smoothed_col = df[col].rolling(n, min_periods=1).mean()
    
    return smoothed_col

In [5]:
def gradient_pace(files, smooth=3):
    data = []
    for file in files:
        if file.endswith('.csv'):
            run_df = get_run_df(file)
            run_df.head()
            data.append({'x': smooth_col(smooth, 'gradient', run_df),
                        'y': smooth_col(smooth, 'inst_speed_meters_sec', run_df),
                        'mode': 'markers',
                        'name': file.replace('_', ' ').replace('.csv','')})
    return data

The cell below does a simple graph of all the runs.

In [35]:
data = gradient_pace(files, 3)

layout = { 'xaxis': {'title': 'Gradient'},
            'yaxis': {'title': 'Speed (meters/sec)'},
            'title': 'Gradient vs Speed'
        }
    

fig = {
    'data': data,
    'layout': layout
    }

py.plot(fig)

'https://plot.ly/~paulsavala/32'

We will now add a dropdown menu to the above graph, to allow the user to select a single run to display. This works if you manually create a dropdown element for each individual run. Obviously this is not ideal, and should be fixed.

In [36]:
updatemenus=list([
    dict(
        buttons=list([   
            dict(
                args=['visible', [True, True, True]],
                label='All',
                method='restyle'
            ),
            dict(
                args=['visible', [True, False, False]],
                label='1',
                method='restyle'
            ),
            dict(
                args=['visible', [False, True, False]],
                label='2',
                method='restyle'
            ),
            dict(
                args=['visible', [False, False, True]],
                label='3',
                method='restyle'
            )    
        ]),
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

#annotations = list([
#    dict(text='Run:', x=0, y=1.085, yref='paper', align='left', showarrow=False)
#])

layout['updatemenus'] = updatemenus
#layout['annotations'] = annotations

fig = {
    'data': data,
    'layout': layout
    }

In [ ]:
py.plot(fig)

In [38]:
def create_visible_list(n):
    '''n is number of runs'''
    false_list = [False for i in np.arange(n)]
    true_i_list = [False for i in np.arange(n)]
    true_i_list[0] = True
    visible_list = []

    for i in np.arange(len(data)):
        visible_list.append(true_i_list[-i:] + true_i_list[:-i])

    visible_list.insert(0,[True for i in np.arange(n)])
    return visible_list

In [39]:
visible_list = create_visible_list(len(data))
buttons = [dict(args=['visible', visible_list[i]], label=str(i), method='restyle') for i in np.arange(len(data)+1)]

In [40]:
updatemenus=list([
    dict(
        buttons=buttons,
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

#annotations = list([
#    dict(text='Run:', x=0, y=1.085, yref='paper', align='left', showarrow=False)
#])

layout['updatemenus'] = updatemenus
#layout['annotations'] = annotations

fig = {
    'data': data,
    'layout': layout
    }

py.plot(fig)

'https://plot.ly/~paulsavala/36'